<div align = "center">

# **Siniestros**

</div>

## Librerias

In [109]:
import pandas as pd

## Data

In [110]:
siniestros = pd.read_csv("../data/in/SiniesCumpli.csv", sep=';')

## Conversión de Tipos de Datos

In [111]:
# Convertir columnas numéricas que están como strings
siniestros['VALOR_ASEGURADO'] = pd.to_numeric(siniestros['VALOR_ASEGURADO'], errors='coerce')
siniestros['RESERVA_ACTUAL_EQUI'] = pd.to_numeric(siniestros['RESERVA_ACTUAL_EQUI'], errors='coerce')
siniestros['PAGOS'] = pd.to_numeric(siniestros['PAGOS'], errors='coerce')

print(f"Dimensiones originales: {siniestros.shape}")
print(f"\nTipos de datos:")
print(siniestros[['VALOR_ASEGURADO', 'RESERVA_ACTUAL_EQUI', 'PAGOS']].dtypes)

Dimensiones originales: (202984, 15)

Tipos de datos:
VALOR_ASEGURADO        float64
RESERVA_ACTUAL_EQUI    float64
PAGOS                  float64
dtype: object


## Eliminación de Columnas Innecesarias

In [112]:
# Definir columnas a eliminar
columnas_eliminar = ['RAMO', 'Conta', 'anio', 'Mes', 'SINIESTRO', 'ULT_FECHA_PAGO']

# Eliminar columnas
siniestros_limpio = siniestros.drop(columns=columnas_eliminar)

print(f"Columnas eliminadas: {columnas_eliminar}")
print(f"\nColumnas restantes: {list(siniestros_limpio.columns)}")
print(f"Dimensiones después de eliminar columnas: {siniestros_limpio.shape}")

Columnas eliminadas: ['RAMO', 'Conta', 'anio', 'Mes', 'SINIESTRO', 'ULT_FECHA_PAGO']

Columnas restantes: ['cod_suc', 'POLIZA', 'DEPARTAMENTO_SINIESTRO', 'VALOR_ASEGURADO', 'FECHA_DE_SINIESTRO', 'FECHA_AVISO', 'AMPARO', 'PAGOS', 'RESERVA_ACTUAL_EQUI']
Dimensiones después de eliminar columnas: (202984, 9)


## Consolidación por POLIZA-AMPARO

In [113]:
# Consolidar por POLIZA-AMPARO
# MAX para valores numéricos (VALOR_ASEGURADO, RESERVA_ACTUAL_EQUI)
# SUM para PAGOS (suma total de todos los pagos históricos)
# FIRST para valores constantes (DEPARTAMENTO, FECHAS, cod_suc)
siniestros_consolidado = siniestros_limpio.groupby(['POLIZA', 'AMPARO']).agg({
    'VALOR_ASEGURADO': 'max',
    'RESERVA_ACTUAL_EQUI': 'max',
    'PAGOS': 'sum',
    'DEPARTAMENTO_SINIESTRO': 'first',
    'FECHA_DE_SINIESTRO': 'first',
    'FECHA_AVISO': 'first',
    'cod_suc': 'first'
}).reset_index()

print(f"Dimensiones consolidadas: {siniestros_consolidado.shape}")
print(f"\nCombinaciones únicas POLIZA-AMPARO: {siniestros_consolidado.shape[0]}")

Dimensiones consolidadas: (5746, 9)

Combinaciones únicas POLIZA-AMPARO: 5746


## Verificación y Validación de Resultados

In [114]:
# Verificar que no hay duplicados
duplicados = siniestros_consolidado.duplicated(subset=['POLIZA', 'AMPARO']).sum()
print(f"Número de duplicados en (POLIZA, AMPARO): {duplicados}")

# Información del dataframe consolidado
print(f"\nInformación del dataframe consolidado:")
print(siniestros_consolidado.info())

# Estadísticas descriptivas de columnas numéricas
print(f"\nEstadísticas de VALOR_ASEGURADO:")
print(siniestros_consolidado['VALOR_ASEGURADO'].describe())

print(f"\nEstadísticas de RESERVA_ACTUAL_EQUI:")
print(siniestros_consolidado['RESERVA_ACTUAL_EQUI'].describe())

print(f"\nEstadísticas de PAGOS:")
print(siniestros_consolidado['PAGOS'].describe())

Número de duplicados en (POLIZA, AMPARO): 0

Información del dataframe consolidado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5746 entries, 0 to 5745
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   POLIZA                  5746 non-null   int64  
 1   AMPARO                  5746 non-null   object 
 2   VALOR_ASEGURADO         5549 non-null   float64
 3   RESERVA_ACTUAL_EQUI     5643 non-null   float64
 4   PAGOS                   5746 non-null   float64
 5   DEPARTAMENTO_SINIESTRO  5746 non-null   object 
 6   FECHA_DE_SINIESTRO      5746 non-null   object 
 7   FECHA_AVISO             5746 non-null   object 
 8   cod_suc                 5746 non-null   int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 404.1+ KB
None

Estadísticas de VALOR_ASEGURADO:
count    5.549000e+03
mean     1.242257e+09
std      5.049634e+09
min      4.191300e+04
25%      1.500000e+07
50%      7.260

In [115]:
# Mostrar primeras filas del dataframe consolidado
siniestros_consolidado.head(10)

,POLIZA,AMPARO,VALOR_ASEGURADO,RESERVA_ACTUAL_EQUI,PAGOS,DEPARTAMENTO_SINIESTRO,FECHA_DE_SINIESTRO,FECHA_AVISO,cod_suc
0,1450,CALIDAD,5.460002e+08,546000205.0,6.093379e+07,13-BOLIVAR,20/06/1999,14/09/1999,11
1,6232,BUEN MANEJO DEL ANTICIPO,2.849548e+08,16893479.0,0.000000e+00,8-ATLANTICO,7/05/1998,7/09/1999,61
2,6296,CUMPLIMIENTO,1.847356e+08,54000000.0,0.000000e+00,68-SANTANDER,20/10/1998,20/10/1998,51
3,10309,PRESTACIONES SOCIALES,1.200000e+09,166481594.0,0.000000e+00,76-VALLE,4/05/1999,10/09/2001,21
4,14213,CUMPLIMIENTO,1.997010e+08,0.0,1.049745e+08,11-BOGOTA D.C.,5/08/2002,15/11/2002,11
5,31225,PRESTACIONES SOCIALES,7.165236e+07,58270562.0,0.000000e+00,23-CORDOBA,25/04/1999,5/03/2003,11
6,62157,DISPOSICIONES LEGALES,1.969916e+07,19699000.0,0.000000e+00,8-ATLANTICO,12/08/1998,21/08/1998,61
7,62557,PRESTACIONES SOCIALES,NaN,42120000.0,0.000000e+00,81-ARAUCA,10/04/2008,28/07/2009,51
8,62830,DISPOSICIONES LEGALES,1.073063e+08,107306000.0,0.000000e+00,8-ATLANTICO,2/10/1998,14/07/1999,61
9,62990,BUEN MANEJO DEL ANTICIPO,2.259262e+08,85029028.0,1.352460e+07,68-SANTANDER,5/10/2007,19/11/2007,51


## Comparación: Antes vs Después

In [116]:
# Comparación de dimensiones
print("=" * 60)
print("COMPARACIÓN: ANTES vs DESPUÉS DE LA LIMPIEZA")
print("=" * 60)

print(f"\nDataframe original:")
print(f"  - Filas: {siniestros.shape[0]:,}")
print(f"  - Columnas: {siniestros.shape[1]}")

print(f"\nDataframe consolidado:")
print(f"  - Filas: {siniestros_consolidado.shape[0]:,}")
print(f"  - Columnas: {siniestros_consolidado.shape[1]}")

print(f"\nReducción de datos:")
print(f"  - Filas eliminadas: {siniestros.shape[0] - siniestros_consolidado.shape[0]:,}")
print(f"  - Porcentaje de reducción: {((siniestros.shape[0] - siniestros_consolidado.shape[0]) / siniestros.shape[0] * 100):.2f}%")
print(f"  - Columnas eliminadas: {siniestros.shape[1] - siniestros_consolidado.shape[1]}")

print(f"\nColumnas finales: {list(siniestros_consolidado.columns)}")

COMPARACIÓN: ANTES vs DESPUÉS DE LA LIMPIEZA

Dataframe original:
  - Filas: 202,984
  - Columnas: 15

Dataframe consolidado:
  - Filas: 5,746
  - Columnas: 9

Reducción de datos:
  - Filas eliminadas: 197,238
  - Porcentaje de reducción: 97.17%
  - Columnas eliminadas: 6

Columnas finales: ['POLIZA', 'AMPARO', 'VALOR_ASEGURADO', 'RESERVA_ACTUAL_EQUI', 'PAGOS', 'DEPARTAMENTO_SINIESTRO', 'FECHA_DE_SINIESTRO', 'FECHA_AVISO', 'cod_suc']


## Carga y Limpieza de Expuestos

In [117]:
# Cargar dataframe de expuestos
expuestos = pd.read_csv('../data/in/ProdCumpli.csv')

print(f"Dimensiones de expuestos: {expuestos.shape}")
print(f"\nColumnas:")
print(expuestos.dtypes)
print(f"\nPrimeras filas:")
expuestos.head()

Dimensiones de expuestos: (1242477, 10)

Columnas:
conta                     int64
anio                      int64
Mes                       int64
Poliza                    int64
CodSucursal               int64
Ramo                     object
VigenciaInicioPoliza     object
VigenciaFinPoliza        object
ValorPrimaEquiv         float64
Amparo                   object
dtype: object

Primeras filas:


,conta,anio,Mes,Poliza,CodSucursal,Ramo,VigenciaInicioPoliza,VigenciaFinPoliza,ValorPrimaEquiv,Amparo
0,1,2025,6,62512,9,CUMPLIMIENTO,09/07/2003,09/10/2030,11544.00,ESTABILIDAD DE LA OBRA
1,2,2025,6,614751,43,CUMPLIMIENTO,10/06/2000,18/07/2078,80000.00,SERIEDAD DE LA OFERTA
2,3,2025,6,1001850,12,CUMPLIMIENTO,05/12/2008,09/02/2029,22000.00,CUMPLIMIENTO
3,4,2025,6,1002709,12,CUMPLIMIENTO,10/07/2009,10/02/2029,22000.00,CUMPLIMIENTO
4,5,2025,6,100038036,10,CUMPLIMIENTO,29/09/2014,29/09/2025,204446.81,CALIDAD Y CORRECTO FUNCIONAMIENTO


In [118]:
# Explorar amparos únicos en expuestos
print("Amparos únicos en expuestos:")
print(expuestos['Amparo'].value_counts())
print(f"\nTotal amparos únicos: {expuestos['Amparo'].nunique()}")

Amparos únicos en expuestos:
Amparo
PRESTACIONES SOCIALES                521197
CUMPLIMIENTO                         212440
CALIDAD DEL SERVICIO                 178505
ESTABILIDAD DE LA OBRA               170329
CALIDAD DE LOS ELEMENTOS              45177
CALIDAD Y CORRECTO FUNCIONAMIENTO     32827
BUEN MANEJO DEL ANTICIPO              31362
SERIEDAD DE LA OFERTA                 19906
CALIDAD                               18881
PAGO ANTICIPADO                        5256
CORRECTO FUNCIONAMIENTO                3290
PROVISION DE REPUESTOS                 1589
DISPOSICIONES LEGALES                  1079
SUMINISTRO DE REPUESTOS                 422
BUEN MANEJO DE MATERIALES               217
Name: count, dtype: int64

Total amparos únicos: 15


In [119]:
# Tomar solo el período más reciente (Junio 2025)
expuestos_limpio = expuestos[
    (expuestos['anio'] == 2025) &
    (expuestos['Mes'] == 6)
].copy()

print(f"Dimensiones antes de filtrar período: {expuestos.shape[0]:,}")
print(f"Dimensiones después (solo Jun 2025): {expuestos_limpio.shape[0]:,}")
print(f"\nCombinaciones únicas Poliza-Amparo: {expuestos_limpio.groupby(['Poliza', 'Amparo']).ngroups:,}")

Dimensiones antes de filtrar período: 1,242,477
Dimensiones después (solo Jun 2025): 1,242,477

Combinaciones únicas Poliza-Amparo: 607,407


## Normalización de Amparos

In [120]:
# Diccionario de mapeo para normalizar amparos truncados
mapeo_amparos = {
    'CALIDAD Y BUEN FUNC.': 'CALIDAD Y CORRECTO FUNCIONAMIENTO',
    'CALIDAD Y CORRECTO FUNCIONAMIE': 'CALIDAD Y CORRECTO FUNCIONAMIENTO',
}

# Aplicar normalización
siniestros_consolidado['AMPARO'] = siniestros_consolidado['AMPARO'].replace(mapeo_amparos)

print("Amparos únicos en siniestros después de normalizar:")
print(siniestros_consolidado['AMPARO'].value_counts())
print(f"\nTotal amparos únicos: {siniestros_consolidado['AMPARO'].nunique()}")

Amparos únicos en siniestros después de normalizar:
AMPARO
CUMPLIMIENTO                         3745
BUEN MANEJO DEL ANTICIPO              731
PRESTACIONES SOCIALES                 540
ESTABILIDAD DE LA OBRA                259
DISPOSICIONES LEGALES                 238
CALIDAD DEL SERVICIO                   60
SERIEDAD DE LA OFERTA                  48
CALIDAD DE LOS ELEMENTOS               36
CALIDAD Y CORRECTO FUNCIONAMIENTO      32
CALIDAD                                28
PAGO ANTICIPADO                        26
CORRECTO FUNCIONAMIENTO                 2
PROVISION DE REPUESTOS                  1
Name: count, dtype: int64

Total amparos únicos: 13


In [121]:
# Comparar amparos entre dataframes
amparos_siniestros = set(siniestros_consolidado['AMPARO'].unique())
amparos_expuestos = set(expuestos_limpio['Amparo'].unique())

print(f"Amparos en siniestros: {len(amparos_siniestros)}")
print(f"Amparos en expuestos: {len(amparos_expuestos)}")
print(f"Amparos en ambos: {len(amparos_siniestros & amparos_expuestos)}")

print(f"\n⚠️ Amparos en siniestros pero NO en expuestos:")
print(amparos_siniestros - amparos_expuestos)

print(f"\n✅ Amparos en expuestos pero NO en siniestros (nunca han siniestrado):")
for amparo in sorted(amparos_expuestos - amparos_siniestros):
    count = expuestos_limpio[expuestos_limpio['Amparo'] == amparo].shape[0]
    print(f"   - {amparo}: {count:,} exposiciones")

Amparos en siniestros: 13
Amparos en expuestos: 15
Amparos en ambos: 13

⚠️ Amparos en siniestros pero NO en expuestos:
set()

✅ Amparos en expuestos pero NO en siniestros (nunca han siniestrado):
   - BUEN MANEJO DE MATERIALES: 217 exposiciones
   - SUMINISTRO DE REPUESTOS: 422 exposiciones


## Join Expuestos-Siniestros (ID Único)

In [122]:
# Crear ID único: POLIZA_AMPARO
siniestros_consolidado['ID_POLIZA_AMPARO'] = (
    siniestros_consolidado['POLIZA'].astype(str) + '_' +
    siniestros_consolidado['AMPARO']
)

expuestos_limpio['ID_POLIZA_AMPARO'] = (
    expuestos_limpio['Poliza'].astype(str) + '_' +
    expuestos_limpio['Amparo']
)

print(f"IDs únicos en siniestros: {siniestros_consolidado['ID_POLIZA_AMPARO'].nunique():,}")
print(f"IDs únicos en expuestos: {expuestos_limpio['ID_POLIZA_AMPARO'].nunique():,}")

IDs únicos en siniestros: 5,746
IDs únicos en expuestos: 607,407


In [123]:
# LEFT JOIN: Expuestos + Siniestros
base_completa = expuestos_limpio.merge(
    siniestros_consolidado,
    on='ID_POLIZA_AMPARO',
    how='left',
    suffixes=('_exp', '_sin')
)

print(f"Dimensiones después del LEFT JOIN: {base_completa.shape}")
print(f"\nRegistros con siniestro: {base_completa['PAGOS'].notna().sum():,}")
print(f"Registros SIN siniestro: {base_completa['PAGOS'].isna().sum():,}")
print(f"Tasa de siniestralidad bruta: {(base_completa['PAGOS'].notna().sum() / base_completa.shape[0] * 100):.2f}%")

Dimensiones después del LEFT JOIN: (1242477, 20)

Registros con siniestro: 7,411
Registros SIN siniestro: 1,235,066
Tasa de siniestralidad bruta: 0.60%


In [ ]:
# Seleccionar y renombrar columnas en un solo paso
# Después del merge tenemos:
# - De expuestos (minúsculas): Poliza, Amparo, CodSucursal, etc.
# - De siniestros (mayúsculas): POLIZA, AMPARO, cod_suc (pandas no aplicó sufijos porque son case-sensitive diferentes)
# Vamos a usar las columnas de EXPUESTOS para identificadores y SINIESTROS para datos del siniestro

base_completa = base_completa[[
    'ID_POLIZA_AMPARO',
    'Poliza',  # De expuestos (renombraremos a POLIZA)
    'Amparo',  # De expuestos (renombraremos a AMPARO)
    'ValorPrimaEquiv',  # De expuestos (renombraremos a EXPOSICION)
    'CodSucursal',  # De expuestos (renombraremos a cod_suc)
    'VigenciaInicioPoliza',  # De expuestos
    'VigenciaFinPoliza',  # De expuestos
    'PAGOS',  # De siniestros
    'RESERVA_ACTUAL_EQUI',  # De siniestros
    'DEPARTAMENTO_SINIESTRO',  # De siniestros
    'FECHA_DE_SINIESTRO',  # De siniestros
    'FECHA_AVISO'  # De siniestros
]].rename(columns={
    'Poliza': 'POLIZA',
    'Amparo': 'AMPARO',
    'ValorPrimaEquiv': 'EXPOSICION',
    'CodSucursal': 'cod_suc',
    'VigenciaInicioPoliza': 'VIGENCIA_INICIO',
    'VigenciaFinPoliza': 'VIGENCIA_FIN'
})

print("Columnas finales en base_completa:")
print(list(base_completa.columns))
print(f"\nDimensiones: {base_completa.shape}")

## Cálculo de Severidad Acotada

In [125]:
# Eliminar filas con EXPOSICION <= 0 o nulo
print(f"Filas antes de filtrar EXPOSICION: {base_completa.shape[0]:,}")

base_filtrada = base_completa[
    (base_completa['EXPOSICION'].notna()) &
    (base_completa['EXPOSICION'] > 0)
].copy()

filas_eliminadas_exp = base_completa.shape[0] - base_filtrada.shape[0]
print(f"Filas eliminadas por EXPOSICION inválida: {filas_eliminadas_exp:,}")
print(f"Filas restantes: {base_filtrada.shape[0]:,}")

Filas antes de filtrar EXPOSICION: 1,242,477
Filas eliminadas por EXPOSICION inválida: 83,342
Filas restantes: 1,159,135


In [126]:
import numpy as np

# Rellenar NaN con 0 en PAGOS y RESERVA (pólizas sin siniestro)
base_filtrada['PAGOS'] = base_filtrada['PAGOS'].fillna(0)
base_filtrada['RESERVA_ACTUAL_EQUI'] = base_filtrada['RESERVA_ACTUAL_EQUI'].fillna(0)

# Calcular SEVERIDAD = min(max(RESERVA, PAGOS), EXPOSICION)
base_filtrada['SEVERIDAD'] = np.minimum(
    np.maximum(
        base_filtrada['RESERVA_ACTUAL_EQUI'],
        base_filtrada['PAGOS']
    ),
    base_filtrada['EXPOSICION']
)

print("Columna SEVERIDAD creada con acotación")
print(f"\nEstadísticas de SEVERIDAD:")
print(base_filtrada['SEVERIDAD'].describe())

print(f"\nDistribución de SEVERIDAD:")
print(f"  - SEVERIDAD = 0: {(base_filtrada['SEVERIDAD'] == 0).sum():,} ({(base_filtrada['SEVERIDAD'] == 0).mean():.2%})")
print(f"  - SEVERIDAD > 0: {(base_filtrada['SEVERIDAD'] > 0).sum():,} ({(base_filtrada['SEVERIDAD'] > 0).mean():.2%})")

Columna SEVERIDAD creada con acotación

Estadísticas de SEVERIDAD:
count    1.159135e+06
mean     1.658983e+04
std      1.668529e+06
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.205759e+09
Name: SEVERIDAD, dtype: float64

Distribución de SEVERIDAD:
  - SEVERIDAD = 0: 1,152,391 (99.42%)
  - SEVERIDAD > 0: 6,744 (0.58%)


In [127]:
# Indicador binario: ¿Tuvo siniestro?
base_filtrada['TUVO_SINIESTRO'] = (base_filtrada['SEVERIDAD'] > 0).astype(int)

print("Indicador TUVO_SINIESTRO creado")
print(f"\nDistribución:")
print(base_filtrada['TUVO_SINIESTRO'].value_counts())
print(f"\nTasa de siniestralidad: {base_filtrada['TUVO_SINIESTRO'].mean():.2%}")

Indicador TUVO_SINIESTRO creado

Distribución:
TUVO_SINIESTRO
0    1152391
1       6744
Name: count, dtype: int64

Tasa de siniestralidad: 0.58%


In [129]:
base_final

,ID_POLIZA_AMPARO,POLIZA,POLIZA,AMPARO,AMPARO,EXPOSICION,DEPARTAMENTO_SINIESTRO,FECHA_DE_SINIESTRO,FECHA_AVISO,cod_suc,VIGENCIA_INICIO,VIGENCIA_FIN,SEVERIDAD,TUVO_SINIESTRO
0,62512_ESTABILIDAD DE LA OBRA,62512,NaN,ESTABILIDAD DE LA OBRA,NaN,11544.00,NaN,NaN,NaN,9,09/07/2003,09/10/2030,0.0,0
1,614751_SERIEDAD DE LA OFERTA,614751,NaN,SERIEDAD DE LA OFERTA,NaN,80000.00,NaN,NaN,NaN,43,10/06/2000,18/07/2078,0.0,0
2,1001850_CUMPLIMIENTO,1001850,NaN,CUMPLIMIENTO,NaN,22000.00,NaN,NaN,NaN,12,05/12/2008,09/02/2029,0.0,0
3,1002709_CUMPLIMIENTO,1002709,NaN,CUMPLIMIENTO,NaN,22000.00,NaN,NaN,NaN,12,10/07/2009,10/02/2029,0.0,0
4,100038036_CALIDAD Y CORRECTO FUNCIONAMIENTO,100038036,NaN,CALIDAD Y CORRECTO FUNCIONAMIENTO,NaN,204446.81,NaN,NaN,NaN,10,29/09/2014,29/09/2025,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242472,100000027_CUMPLIMIENTO,100000027,NaN,CUMPLIMIENTO,NaN,966517.81,NaN,NaN,NaN,92,12/05/2025,11/05/2033,0.0,0
1242473,100000027_PRESTACIONES SOCIALES,100000027,NaN,PRESTACIONES SOCIALES,NaN,966517.81,NaN,NaN,NaN,92,12/05/2025,11/05/2033,0.0,0
1242474,100000054_CALIDAD DEL SERVICIO,100000054,NaN,CALIDAD DEL SERVICIO,NaN,59205.37,NaN,NaN,NaN,52,05/06/2025,05/06/2030,0.0,0
1242475,100000054_CUMPLIMIENTO,100000054,NaN,CUMPLIMIENTO,NaN,59205.37,NaN,NaN,NaN,52,05/06/2025,05/06/2030,0.0,0


In [128]:
# Eliminar PAGOS y RESERVA_ACTUAL_EQUI (ya consolidadas en SEVERIDAD)
base_final = base_filtrada.drop(columns=['PAGOS', 'RESERVA_ACTUAL_EQUI'])

print("Columnas eliminadas: ['PAGOS', 'RESERVA_ACTUAL_EQUI']")
print(f"Dimensiones finales: {base_final.shape}")
print(f"\nColumnas finales:")
for i, col in enumerate(base_final.columns, 1):
    print(f"  {i}. {col} ({base_final[col].dtype})")

Columnas eliminadas: ['PAGOS', 'RESERVA_ACTUAL_EQUI']
Dimensiones finales: (1159135, 14)

Columnas finales:
  1. ID_POLIZA_AMPARO (object)


AttributeError: 'DataFrame' object has no attribute 'dtype'

## Dataframes Finales para Análisis de Credibilidad

In [ ]:
# Dataframe solo con pólizas-amparos siniestrados
base_solo_siniestros = base_final[base_final['SEVERIDAD'] > 0].copy()

print(f"Base completa (incluye no siniestrados): {base_final.shape[0]:,} registros")
print(f"Base solo siniestrados: {base_solo_siniestros.shape[0]:,} registros")
print(f"\nTasa de siniestralidad: {(base_solo_siniestros.shape[0] / base_final.shape[0] * 100):.2f}%")

In [ ]:
# Resumen completo de toda la transformación
print("=" * 70)
print("RESUMEN COMPLETO DE TRANSFORMACIÓN")
print("=" * 70)

print(f"\n1. SINIESTROS (SiniesCumpli.csv)")
print(f"   - Filas originales: 202,984")
print(f"   - Filas consolidadas (POLIZA-AMPARO): {siniestros_consolidado.shape[0]:,}")

print(f"\n2. EXPUESTOS (ProdCumpli.csv)")
print(f"   - Filas originales: 1,242,477")
print(f"   - Filas Jun 2025: {expuestos_limpio.shape[0]:,}")

print(f"\n3. BASE COMPLETA (JOIN)")
print(f"   - Filas después de join: {base_completa.shape[0]:,}")
print(f"   - Filas después de filtrar exposición: {base_final.shape[0]:,}")

print(f"\n4. DISTRIBUCIÓN FINAL")
print(f"   - Pólizas-amparos sin siniestro: {(base_final['SEVERIDAD'] == 0).sum():,}")
print(f"   - Pólizas-amparos con siniestro: {(base_final['SEVERIDAD'] > 0).sum():,}")
print(f"   - Tasa de siniestralidad: {base_final['TUVO_SINIESTRO'].mean():.2%}")

print(f"\n5. SEVERIDAD (solo siniestrados)")
print(f"   - Promedio: ${base_solo_siniestros['SEVERIDAD'].mean():,.2f}")
print(f"   - Mediana: ${base_solo_siniestros['SEVERIDAD'].median():,.2f}")
print(f"   - Máximo: ${base_solo_siniestros['SEVERIDAD'].max():,.2f}")

print(f"\n6. COLUMNAS FINALES ({base_final.shape[1]})")
for i, col in enumerate(base_final.columns, 1):
    print(f"   {i}. {col}")

In [ ]:
# Mostrar muestras
print("Muestra de BASE_FINAL (incluye no siniestrados):")
print(base_final.head(10))

print("\nMuestra de BASE_SOLO_SINIESTROS (solo siniestrados):")
print(base_solo_siniestros.head(10))